In [1]:
import numpy as np
import pandas as pd
import keras
import keras.backend as kb
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

# Keras specific
import keras
from keras.models import Sequential
from keras.layers import Dense

df = pd.read_csv('Datasets/covid_19_india.csv')

df.head()

Using TensorFlow backend.


,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed
0,1,30/01/20,6:00 PM,Kerala,1,0,0,0,1
1,2,31/01/20,6:00 PM,Kerala,1,0,0,0,1
2,3,01/02/20,6:00 PM,Kerala,2,0,0,0,2
3,4,02/02/20,6:00 PM,Kerala,3,0,0,0,3
4,5,03/02/20,6:00 PM,Kerala,3,0,0,0,3


In [2]:
df['Date'] = pd.to_datetime(df.Date)
df['date_delta'] = (df['Date'] - df['Date'].min())  / np.timedelta64(1,'D')
df['Time'] = pd.to_datetime(df.Time)
df['time_delta'] = (df['Time'] - df['Time'].min())  / np.timedelta64(1,'D')


df.head()

,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed,date_delta,time_delta
0,1,2020-01-30,2020-04-29 18:00:00,Kerala,1,0,0,0,1,28.0,0.333333
1,2,2020-01-31,2020-04-29 18:00:00,Kerala,1,0,0,0,1,29.0,0.333333
2,3,2020-01-02,2020-04-29 18:00:00,Kerala,2,0,0,0,2,0.0,0.333333
3,4,2020-02-02,2020-04-29 18:00:00,Kerala,3,0,0,0,3,31.0,0.333333
4,5,2020-03-02,2020-04-29 18:00:00,Kerala,3,0,0,0,3,60.0,0.333333


In [3]:
data = df.values
X = data[:,9:10]
y = data[:,8]



X=np.array(X)
y = np.array(y)

X


array([[28.0],
       [29.0],
       [0.0],
       ...,
       [117.0],
       [117.0],
       [117.0]], dtype=object)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)
X_test

array([[87.0],
       [62.0],
       [102.0],
       [113.0],
       [116.0],
       [123.0],
       [184.0],
       [84.0],
       [109.0],
       [246.0],
       [88.0],
       [117.0],
       [111.0],
       [82.0],
       [102.0],
       [105.0],
       [74.0],
       [84.0],
       [87.0],
       [109.0],
       [62.0],
       [85.0],
       [154.0],
       [106.0],
       [62.0],
       [116.0],
       [89.0],
       [337.0],
       [106.0],
       [122.0],
       [123.0],
       [33.0],
       [275.0],
       [113.0],
       [102.0],
       [110.0],
       [88.0],
       [83.0],
       [78.0],
       [115.0],
       [246.0],
       [245.0],
       [122.0],
       [307.0],
       [184.0],
       [153.0],
       [85.0],
       [336.0],
       [88.0],
       [93.0],
       [154.0],
       [114.0],
       [275.0],
       [109.0],
       [106.0],
       [109.0],
       [306.0],
       [62.0],
       [116.0],
       [154.0],
       [92.0],
       [307.0],
       [117.0],
       [80.0]

In [5]:
model = Sequential()
model.add(Dense(500, input_dim=2, activation= "relu"))
model.add(Dense(100, activation= "relu"))
model.add(Dense(50, activation= "relu"))
model.add(Dense(1))

In [6]:
optimizer = tf.keras.optimizers.RMSprop(0.0099)
model.compile(loss='mean_squared_error',optimizer=optimizer)
model.fit(X_train,y_train,epochs=20)

ValueError: Error when checking input: expected dense_1_input to have shape (2,) but got array with shape (1,)

In [ ]:
predict=model.predict(X_test)

print(predict)
print(y)


In [ ]:
limsx = [0, 400]
limsy = [0, 800]
plt.xlim(limsx)
plt.ylim(limsy)

plt.plot()
plt.plot(X_test, predict)